# Relation Graph Generator

This notebook builds an interactive HTML graph of all variables (nodes) and relations (edges).
It uses the relation registry only (no reactor defaults).


In [1]:
from pathlib import Path
import sys
import json
import hashlib
import html

root = Path.cwd()
if not (root / 'src' / 'fusdb').is_dir():
    for parent in root.parents:
        if (parent / 'src' / 'fusdb').is_dir():
            root = parent
            break
sys.path.insert(0, str(root / 'src'))

from fusdb import relations
from fusdb.relation_class import _RELATION_REGISTRY
from fusdb.registry import load_allowed_variables
from fusdb.variable_class import Variable

relations.import_relations()
allowed_vars, _, _ = load_allowed_variables()
relations_list = list(_RELATION_REGISTRY)

def color_for(name: str) -> str:
    h = hashlib.md5(name.encode('utf-8')).hexdigest()
    return f'#{h[:6]}'

def _format_value(value: object) -> str:
    if value is None:
        return "None"
    if isinstance(value, dict):
        if not value:
            return "{}"
        return ", ".join(f"{k}: {_format_value(v)}" for k, v in value.items())
    if isinstance(value, (list, tuple, set)):
        if not value:
            return "[]"
        return ", ".join(_format_value(v) for v in value)
    if callable(value):
        return f"{value.__module__}.{value.__name__}"
    return str(value)

def _html_section(title: str, items: list[tuple[str, object]]) -> str:
    lines = [f"<b>{html.escape(title)}</b>"]
    for key, value in items:
        lines.append(f"<b>{html.escape(str(key))}</b>: {html.escape(_format_value(value))}")
    return "<br>".join(lines)

# Collect variable names from registry + relations
var_names = list(allowed_vars.keys())
seen = set(var_names)
for rel in relations_list:
    for name in rel.variables:
        if name not in seen:
            seen.add(name)
            var_names.append(name)

def var_detail_html(name: str) -> str:
    spec = allowed_vars.get(name, {}) or {}
    var = Variable(name=name, unit=spec.get("default_unit"))
    var_items = [
        ("name", var.name),
        ("unit", var.unit),
        ("rel_tol", var.rel_tol),
        ("abs_tol", var.abs_tol),
        ("method", var.method),
        ("input_source", var.input_source),
        ("fixed", var.fixed),
        ("values", var.values),
        ("value_passes", var.value_passes),
        ("history", var.history),
        ("current_value", var.current_value),
        ("input_value", var.input_value),
    ]
    detail = _html_section("Variable", var_items)
    if spec:
        reg_items = []
        for key in ("default_unit", "aliases", "constraints", "description"):
            if key in spec:
                reg_items.append((key, spec.get(key)))
        if reg_items:
            detail += "<br><br>" + _html_section("Registry", reg_items)
    else:
        detail += "<br><br>" + _html_section("Registry", [("status", "not in allowed_variables.yaml")])
    return detail

def relation_detail_html(rel) -> str:
    def _fn_label(fn):
        return f"{fn.__module__}.{fn.__name__}" if fn else "None"
    initial = {k: _fn_label(v) for k, v in (rel.initial_guesses or {}).items()}
    solve_for = {k: _fn_label(v) for k, v in (rel.solve_for or {}).items()} if rel.solve_for else {}
    rel_items = [
        ("name", rel.name),
        ("output", rel.output),
        ("inputs", list(rel.inputs)),
        ("variables", list(rel.variables)),
        ("tags", list(rel.tags or ())),
        ("constraints", list(rel.constraints or ())),
        ("rel_tol_default", rel.rel_tol_default),
        ("abs_tol_default", rel.abs_tol_default),
        ("initial_guesses", initial),
        ("solve_for", solve_for),
        ("func", _fn_label(rel.func)),
        ("sympy_expr", str(rel.sympy_expr) if rel.sympy_expr is not None else None),
        ("_sympy_symbols", list(getattr(rel, "_sympy_symbols", {}) or {})),
        ("_inverse_cache", list(getattr(rel, "_inverse_cache", {}) or {})),
    ]
    return _html_section("Relation", rel_items)

nodes = []
for name in var_names:
    detail_html = var_detail_html(name)
    nodes.append({
        'id': name,
        'label': name,
        'title': detail_html,
        'detail_html': detail_html,
        'shape': 'dot',
        'color': '#97c2fc',
        'search_blob': ' '.join([name] + [str(v) for v in (allowed_vars.get(name, {}) or {}).values()]).lower(),
    })

edges = []
relation_meta = []
for rel in relations_list:
    rel_name = rel.name
    rel_info = {
        'name': rel_name,
        'output': rel.output,
        'inputs': list(rel.inputs),
        'tags': list(rel.tags or ()),
        'constraints': list(rel.constraints or ()),
        'rel_tol_default': rel.rel_tol_default,
        'abs_tol_default': rel.abs_tol_default,
        'initial_guesses': sorted((rel.initial_guesses or {}).keys()),
        'solve_for': sorted((rel.solve_for or {}).keys()) if rel.solve_for else [],
        'has_sympy_expr': rel.sympy_expr is not None,
    }
    relation_meta.append(rel_info)
    detail_html = relation_detail_html(rel)

    color = color_for(rel_name)
    for inp in rel.inputs:
        edges.append({
            'from': inp,
            'to': rel.output,
            'label': rel_name,
            'title': detail_html,
            'detail_html': detail_html,
            'relation': rel_name,
            'arrows': 'to',
            'color': color,
            'search_blob': ' '.join(str(v) for v in rel_info.values()).lower(),
        })

# Build suggestions: names first, then other searchable tokens
name_suggestions = sorted({*var_names, *[r['name'] for r in relation_meta]})
other_tokens = []
for name, spec in allowed_vars.items():
    for alias in spec.get('aliases', []) or []:
        other_tokens.append(alias)
for rel in relation_meta:
    other_tokens += rel.get('tags', [])
    other_tokens.append(rel.get('output'))
other_suggestions = sorted({t for t in other_tokens if t})
suggestions = name_suggestions + [s for s in other_suggestions if s not in name_suggestions]

html = f'''<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>Relation graph</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/vis-network/9.1.2/dist/vis-network.min.css" crossorigin="anonymous" referrerpolicy="no-referrer" />
    <script src="https://cdnjs.cloudflare.com/ajax/libs/vis-network/9.1.2/dist/vis-network.min.js" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
    <style>
      body {{ font-family: sans-serif; margin: 0; }}
      #toolbar {{ padding: 12px; border-bottom: 1px solid #ddd; display: flex; gap: 8px; align-items: center; background: #fafafa; }}
      #search {{ width: 360px; padding: 6px 8px; }}
      #layout {{ display: grid; grid-template-columns: minmax(0, 1fr) 360px; }}
      #mynetwork {{ width: 100%; height: 800px; border: 1px solid #ddd; background: #fff; }}
      #details {{ padding: 10px 12px; font-size: 13px; border-left: 1px solid #ddd; background: #fafafa; overflow: auto; height: 800px; }}
      #details .title {{ font-weight: 600; margin-bottom: 6px; }}
      #details .empty {{ color: #888; }}
      .hint {{ color: #666; font-size: 12px; }}
      @media (max-width: 900px) {{
        #layout {{ grid-template-columns: 1fr; }}
        #mynetwork {{ height: 70vh; border-bottom: 1px solid #ddd; }}
        #details {{ border-left: none; border-top: 1px solid #ddd; height: auto; min-height: 220px; }}
      }}
    </style>
  </head>
  <body>
    <div id="toolbar">
      <input id="search" list="search-list" placeholder="Search variables or relations" />
      <datalist id="search-list">
      </datalist>
      <span class="hint">Matches names first, otherwise all fields.</span>
    </div>
    <div id="layout">
      <div id="mynetwork"></div>
      <div id="details">
        <div class="title">Details</div>
        <div id="details-body"><span class="empty">Hover a node/edge to see details.</span></div>
      </div>
    </div>
    <script>
      const details = document.getElementById('details-body');
      function setDetails(html) {{
        details.innerHTML = html || '<span class="empty">Hover a node/edge to see details.</span>';
      }}

      if (!window.vis) {{
        setDetails('<span class="empty">vis-network failed to load. Open this HTML in a browser or allow external resources.</span>');
      }} else {{
        const nodes = new vis.DataSet({json.dumps(nodes)});
        const edges = new vis.DataSet({json.dumps(edges)});
        const suggestions = {json.dumps(suggestions)};
        const data = {{ nodes, edges }};
        const options = {{
          nodes: {{ shape: 'dot', size: 18, font: {{ size: 14, face: 'monospace' }} }},
          edges: {{ arrows: {{ to: {{ enabled: true }} }}, font: {{ size: 12, align: 'middle' }} }},
          interaction: {{ hover: true }},
          physics: {{ barnesHut: {{ springLength: 140, springConstant: 0.03 }} }}
        }};
        const container = document.getElementById('mynetwork');
        const network = new vis.Network(container, data, options);
        setTimeout(() => {{
          try {{
            network.fit({{ animation: false }});
          }} catch (err) {{
            // ignore
          }}
        }}, 0);

        const datalist = document.getElementById('search-list');
        datalist.innerHTML = suggestions.map(s => `<option value="${{s}}"></option>`).join('');

        const nodeData = nodes.get({{ returnType: 'Object' }});
        const edgeData = edges.get({{ returnType: 'Object' }});
        let lastQuery = '';
        let pinned = null;

        function getNodeDetail(node) {{
          return (node && (node.detail_html || node.title)) || '';
        }}
        function getEdgeDetail(edge) {{
          return (edge && (edge.detail_html || edge.title)) || '';
        }}
        setDetails(`<span class="empty">Loaded ${{nodes.getIds().length}} nodes and ${{edges.getIds().length}} edges. Hover a node/edge to see details.</span>`);
        const nodeColors = {{}};
        const edgeColors = {{}};
        Object.keys(nodeData).forEach(id => nodeColors[id] = nodeData[id].color);
        Object.keys(edgeData).forEach(id => edgeColors[id] = edgeData[id].color);

        function resetColors() {{
          Object.keys(nodeData).forEach(id => nodes.update({{ id, color: nodeColors[id], font: {{ color: '#222' }} }}));
          Object.keys(edgeData).forEach(id => edges.update({{ id, color: edgeColors[id], font: {{ color: '#222' }} }}));
        }}

        function highlightRelation(relName) {{
          if (!relName) {{
            applySearch(lastQuery);
            return;
          }}
          Object.keys(edgeData).forEach(id => {{
            const edge = edgeData[id];
            const hit = edge.relation === relName;
            edges.update({{ id, color: hit ? edgeColors[id] : '#e0e0e0', font: {{ color: hit ? '#000' : '#999' }} }});
          }});
        }}

        function applySearch(query) {{
          lastQuery = query;
          const raw = query.trim();
          const q = raw.toLowerCase();
          if (!raw) {{
            resetColors();
            return;
          }}

          let exactNode = nodeData[raw];
          let exactEdge = Object.values(edgeData).find(e => (e.relation || '') === raw);
          if (!exactNode && !exactEdge) {{
            exactNode = Object.values(nodeData).find(n => (n.id || '').toLowerCase() === q);
            exactEdge = Object.values(edgeData).find(e => (e.relation || '').toLowerCase() === q);
          }}

          const nodeMatches = [];
          const edgeMatches = [];

          if (exactNode) {{
            nodeMatches.push(exactNode.id);
          }} else if (exactEdge) {{
            edgeMatches.push(exactEdge.id);
          }} else {{
            Object.values(nodeData).forEach(n => {{
              if ((n.search_blob || '').includes(q)) nodeMatches.push(n.id);
            }});
            Object.values(edgeData).forEach(e => {{
              if ((e.search_blob || '').includes(q)) edgeMatches.push(e.id);
            }});
          }}

          Object.keys(nodeData).forEach(id => {{
            const hit = nodeMatches.includes(id);
            nodes.update({{ id, color: hit ? '#ffa500' : '#e0e0e0', font: {{ color: hit ? '#000' : '#999' }} }});
          }});
          Object.keys(edgeData).forEach(id => {{
            const hit = edgeMatches.includes(id);
            edges.update({{ id, color: hit ? '#ff5a5a' : '#e0e0e0', font: {{ color: hit ? '#000' : '#999' }} }});
          }});

          if (nodeMatches.length) {{
            network.selectNodes(nodeMatches);
            network.focus(nodeMatches[0], {{ scale: 1.2 }});
          }} else if (edgeMatches.length) {{
            network.selectEdges(edgeMatches);
          }}
        }}

        network.on('hoverNode', (params) => {{
          if (pinned) return;
          const n = nodeData[params.node];
          if (n) setDetails(getNodeDetail(n));
        }});
        network.on('hoverEdge', (params) => {{
          if (pinned) return;
          const e = edgeData[params.edge];
          if (e) {{
            setDetails(getEdgeDetail(e));
            highlightRelation(e.relation);
          }}
        }});
        network.on('blurNode', () => {{
          if (pinned) return;
          setDetails(`<span class="empty">Loaded ${{nodes.getIds().length}} nodes and ${{edges.getIds().length}} edges. Hover a node/edge to see details.</span>`);
          applySearch(lastQuery);
        }});
        network.on('blurEdge', () => {{
          if (pinned) return;
          setDetails(`<span class="empty">Loaded ${{nodes.getIds().length}} nodes and ${{edges.getIds().length}} edges. Hover a node/edge to see details.</span>`);
          applySearch(lastQuery);
        }});
        network.on('click', (params) => {{
          if (params.nodes.length) {{
            const n = nodeData[params.nodes[0]];
            if (n) {{
              pinned = {{ type: 'node', id: n.id }};
              setDetails(getNodeDetail(n));
              applySearch(lastQuery);
            }}
            return;
          }}
          if (params.edges.length) {{
            const e = edgeData[params.edges[0]];
            if (e) {{
              pinned = {{ type: 'relation', name: e.relation }};
              setDetails(getEdgeDetail(e));
              highlightRelation(e.relation);
            }}
            return;
          }}
          pinned = null;
          setDetails(`<span class="empty">Loaded ${{nodes.getIds().length}} nodes and ${{edges.getIds().length}} edges. Hover a node/edge to see details.</span>`);
          applySearch(lastQuery);
        }});
        const searchInput = document.getElementById('search');
        searchInput.addEventListener('input', (e) => {{
          pinned = null;
          applySearch(e.target.value);
        }});
      }}
    </script>
  </body>
</html>
'''

out_path = root / 'docs' / 'relations_variables_graph.html'
out_path.write_text(html, encoding='utf-8')
print(f'Wrote: {out_path}')


Wrote: /home/alessmor/Scrivania/fusdb/docs/relations_variables_graph.html
